In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
import re
from bs4 import BeautifulSoup
import contractions
from sklearn.model_selection import train_test_split
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vibhu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vibhu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [1]:
pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 11.0 MB/s eta 0:00:00


In [ ]:
! pip install bs4 # in case you don't have it installed

# Dataset: https://web.archive.org/web/20201127142707if_/https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz

## Read Data

In [2]:
url = "amazon_reviews_us_Office_Products_v1_00.tsv"
data = pd.read_csv(url,index_col = False,sep='\t+', on_bad_lines='skip')

C:\Users\vibhu\AppData\Local\Temp\ipykernel_35724\3782226354.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv(url,index_col = False,sep='\t+', on_bad_lines='skip')


## Keep Reviews and Ratings

In [3]:
trimmedData = data[['star_rating','review_body']]

 ## We form two classes and select 50000 reviews randomly from each class.



In [4]:
trimmedData['star_rating'].unique()

array([5, 1, 4, 2, 3], dtype=int64)

In [5]:
trimmedData.loc[trimmedData['star_rating'] <= 3 , 'star_rating'] = 1
trimmedData.loc[trimmedData['star_rating'] >= 4,'star_rating'] = 2

Assign class as 1 for rating 1,2,3 and class 2 for rating above 3

In [9]:
data1 = trimmedData[trimmedData['star_rating'] ==1 ].sample(n=50000, random_state=49)
data2 = trimmedData[trimmedData['star_rating'] == 2].sample(n=50000, random_state=49)

finalData = pd.concat([data1,data2])

Randomly select 50000 records from each class and concat it to make final dataset

# Data Cleaning



# Pre-processing

In [10]:
def remove_html_tags(text):
    soup = BeautifulSoup(text, 'html.parser')
    return soup.get_text()

def extend_contractions(text):
    extended_words = []
    for w in text.split():
        extended_words.append(contractions.fix(w))
    return ' '.join(extended_words)

def text_cleaning(text):
    soup = BeautifulSoup(text, 'html.parser')
    #Remove html tags
    text = soup.get_text()
    #remove URLs
    text = re.sub(r'http\S+', '', text, flags=re.MULTILINE)
    #Extend contractions
    text = contractions.fix(text)
    #Convert lower case
    text = text.lower()
    #Remove punctuations and numbers
    text = re.sub(r'[^a-zA-Z\s]','',text)
    return text


Remove HTML tags,URL, punctuations and numbers. convert to lower case and extened constractions

In [12]:
finalData = finalData.drop(finalData[finalData['review_body'].isna()].index)
print("NaN data: "+ str(len(finalData[finalData['review_body'].isna()])))

NaN data: 0


Check for Nan records and remove them from data

In [13]:
finalData['cleaned_review_body'] = finalData['review_body'].apply(text_cleaning)

C:\Users\vibhu\AppData\Local\Temp\ipykernel_35724\3713851753.py:12: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')


Apply text cleaning

In [14]:
avg_len_before = finalData['review_body'].str.len().mean()
avg_len_after = finalData['cleaned_review_body'].str.len().mean()

print(str(avg_len_before) + ' ' + str(avg_len_after))

315.80036800368003 302.4426644266443


Removed URL, html tags, etended contractions and removed punctuations and numbers from reviews. That lead to decrease in average length of reviews

## remove the stop words

In [15]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))

def remove_stopWords(text):
    word_tokens = text.split(' ')
    filtered_words = [x for x in word_tokens if not x.lower() in stop_words]
    return ' '.join(filtered_words)


In [16]:
finalData['processed_review'] = finalData['cleaned_review_body'].apply(remove_stopWords)

Remove stopwords using nltk.corpus and applied it on the data

## perform lemmatization  

In [13]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vibhu\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [14]:
from nltk.stem import WordNetLemmatizer
def lemmatize_text(text):
    lemmatizer = nltk.stem.WordNetLemmatizer()
    word_tokens = nltk.tokenize.word_tokenize(text)
    for i in range(len(word_tokens)):
        word_tokens[i] = lemmatizer.lemmatize(word_tokens[i])
    return ' '.join(word_tokens)

In [15]:
finalData['processed_review'] = finalData['processed_review'].apply(lemmatize_text)

In [17]:
avg_len_after_process = finalData['processed_review'].str.len().mean()
avg_len_before_process = finalData['cleaned_review_body'].str.len().mean()

print(str(avg_len_before_process)+" , " + str(avg_len_after_process))

302.4426644266443 , 191.89438894388945


Removed stop words and performed lemmatization to convert words to normal form. Ex. tried -> try, better -> good

significantly decreased the average lenth of review

# TF-IDF and BoW Feature Extraction

In [18]:
X = finalData['review_body']
y = finalData['star_rating']
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state = 49)

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
bowVectorizer = CountVectorizer()
x_train_bow = bowVectorizer.fit_transform(x_train)
x_test_bow = bowVectorizer.transform(x_test)

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
x_train_tfidf = tfidf_vectorizer.fit_transform(x_train)
x_test_tfidf = tfidf_vectorizer.transform(x_test)

Featurer extraction using bag of words and TF-IDF. Transformed data in the format which can be used in the models

# Perceptron Using Both Features

In [20]:
y_train = y_train.map({1: -1, 2: 1})
y_test = y_test.map({1: -1, 2:1})

In [21]:
from sklearn.linear_model import Perceptron
from sklearn.metrics import precision_score,recall_score,f1_score
bowPerceptron = Perceptron()
bowPerceptron.fit(x_train_bow,y_train)
y_pred_bow = bowPerceptron.predict(x_test_bow)
precision_bow = precision_score(y_test,y_pred_bow)
recall_bow = recall_score(y_test, y_pred_bow)
f1_bow = f1_score(y_test,y_pred_bow)

tfidfPerceptron = Perceptron()
tfidfPerceptron.fit(x_train_tfidf,y_train)
y_pred_tfidf = tfidfPerceptron.predict(x_test_tfidf)
precision_tfidf = precision_score(y_test, y_pred_tfidf)
recall_tfidf = recall_score(y_test,y_pred_tfidf)
f1_tfidf = f1_score(y_test,y_pred_tfidf)

In [22]:
print("BoW")
print(str(precision_bow) + " " + str(recall_bow) + " " + str(f1_bow))
print("Tf-IDF")
print(str(precision_tfidf) + " " + str(recall_tfidf) + " " + str(f1_tfidf))

BoW
0.8157245827010622 0.8638144019282916 0.8390810204380275
Tf-IDF
0.7753077347051258 0.847644872953701 0.8098642230005277


Trained perceptron using bow and TF-IDF data and got evalution metrics. Tested using the test split. BoW gives better precision with perceptron

# SVM Using Both Features

In [23]:
from sklearn.svm import LinearSVC
bowSVC = LinearSVC()
bowSVC.fit(x_train_bow,y_train)
y_pred_bow = bowSVC.predict(x_test_bow)
precision_bow = precision_score(y_test,y_pred_bow)
recall_bow = recall_score(y_test, y_pred_bow)
f1_bow = f1_score(y_test,y_pred_bow)

tfidfSVC = LinearSVC()
tfidfSVC.fit(x_train_tfidf,y_train)
y_pred_tfidf = tfidfSVC.predict(x_test_tfidf)
precision_tfidf = precision_score(y_test, y_pred_tfidf)
recall_tfidf = recall_score(y_test,y_pred_tfidf)
f1_tfidf = f1_score(y_test,y_pred_tfidf)

C:\Users\vibhu\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\vibhu\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\vibhu\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


In [24]:
print("BoW")
print(str(precision_bow) + " " + str(recall_bow) + " " + str(f1_bow))
print("Tf-IDF")
print(str(precision_tfidf) + " " + str(recall_tfidf) + " " + str(f1_tfidf))

BoW
0.8397222765499707 0.8624083559305011 0.8509141356587226
Tf-IDF
0.8669432358595568 0.8605001506477855 0.8637096774193549


TF-IDF gives better result on test split when used with SVM

# Logistic Regression Using Both Features

In [25]:
from sklearn.linear_model import LogisticRegression
bowReg = LogisticRegression()
bowReg.fit(x_train_bow,y_train)
y_pred_bow = bowReg.predict(x_test_bow)
precision_bow = precision_score(y_test,y_pred_bow)
recall_bow = recall_score(y_test, y_pred_bow)
f1_bow = f1_score(y_test,y_pred_bow)

tfidfReg = LogisticRegression()
tfidfReg.fit(x_train_tfidf,y_train)
y_pred_tfidf = tfidfReg.predict(x_test_tfidf)
precision_tfidf_reg = precision_score(y_test, y_pred_tfidf)
recall_tfidf_reg = recall_score(y_test,y_pred_tfidf)
f1_tfidf_reg = f1_score(y_test,y_pred_tfidf)

C:\Users\vibhu\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\vibhu\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/st

In [26]:
print("BoW")
print(str(precision_bow) + " " + str(recall_bow) + " " + str(f1_bow))
print("TF-IDF")
print(str(precision_tfidf_reg) + " " + str(recall_tfidf_reg) + " " + str(f1_tfidf_reg))

BoW
0.8502415458937198 0.8838003414683138 0.8666962131284778
TF-IDF
0.8769938650306749 0.8614040373606509 0.8691290469676243


Gives similar results as SVM for TFIDF. Works better than SVM for BoW

# Naive Bayes Using Both Features

In [27]:
from sklearn.naive_bayes import MultinomialNB
bowNaive = MultinomialNB()
bowNaive.fit(x_train_bow,y_train)
y_pred_bow = bowNaive.predict(x_test_bow)
precision_bow = precision_score(y_test,y_pred_bow)
recall_bow = recall_score(y_test, y_pred_bow)
f1_bow = f1_score(y_test,y_pred_bow)

tfidfNaive = MultinomialNB()
tfidfNaive.fit(x_train_tfidf,y_train)
y_pred_tfidf = tfidfNaive.predict(x_test_tfidf)
precision_tfidf = precision_score(y_test, y_pred_tfidf)
recall_tfidf = recall_score(y_test,y_pred_tfidf)
f1_tfidf = f1_score(y_test,y_pred_tfidf)

In [28]:
print("BoW")
print(str(precision_bow) + " " + str(recall_bow) + " " + str(f1_bow))
print("Tf-IDF")
print(str(precision_tfidf) + " " + str(recall_tfidf) + " " + str(f1_tfidf))

BoW
0.8032786885245902 0.8661243346389474 0.833518581162712
Tf-IDF
0.8540012872774083 0.7995380134578688 0.8258727112402096


Gives better results for BoW than TF-IDF